In [1]:
import pandas as pd
import ast
import re

In [3]:
df=pd.read_excel("hyderabad_cars.xlsx")

CAR DETAIL

In [4]:

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])
new_car_details_df.insert(0, 'state', 'hyderabad')

In [5]:
new_car_details_df.drop(columns=["it","owner","priceFixedText","priceActual","priceSaving","trendingText.imgUrl",
                          "trendingText.heading","trendingText.desc"],inplace=True)

def convert_numberic(value):
    new=value.replace("₹","").replace(',', '').strip()
    if "Lakh" in new:
        return float(new.replace('Lakh', '').strip())*1e5
    elif "Crore" in new:
        return float(new.replace("Crore","").strip())*1e7
    else:
        return float(new)
new_car_details_df["price"]=new_car_details_df["price"].apply(convert_numberic)

In [6]:

# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)


In [7]:

def extract_year(value):
    match = re.search(r'\b\d{4}\b', str(value))
    if match:
        # Extract the year and return it as an integer
        return int(match.group())
    return pd.NA  # Use pd.NA for missing values

# Apply the function to extract the year
car_overview['Registration Year'] = car_overview['Registration Year'].apply(extract_year)

# Convert the column datatype to 'Int64' (nullable integer type)
car_overview['Registration Year'] = car_overview['Registration Year'].astype('Int64')


def extract_number(text):
    # Ensure the input is a string
    if isinstance(text, str):
        match = re.search(r'\d+', text)
        if match:
            return int(match.group())
    return None


car_overview['Seats'] = car_overview['Seats'].apply(extract_number)


import pandas as pd
import re

def extract_kms(text):
    
    if isinstance(text, str):
        
        numeric_value = re.sub(r'[^\d,]', '', text)
        
        return int(numeric_value.replace(',', ''))
    return None


# Apply the function to the 'Distance' column
car_overview['Kms Driven'] = car_overview['Kms Driven'].apply(extract_kms)
car_overview.drop(columns=["Ownership"],inplace=True)

In [8]:
car_overview["Registration Year"]=car_overview["Registration Year"].fillna(2019)
car_overview["Seats"]=car_overview["Seats"].fillna(5.0)
car_overview["Kms Driven"]=car_overview["Kms Driven"].fillna(car_overview["Kms Driven"].mean())
car_overview["Year of Manufacture"]=car_overview["Year of Manufacture"].fillna(2019)

CAR FEATURES

In [9]:
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
result=[]
for sec in df["new_car_feature"]:
   
   
    df1 = pd.DataFrame([item['value'] for item in sec["top"]], columns=['Features'])

# Concatenate all the values into a single string and place it in a single row
    concatenated_features = ','.join(df1['Features'])
    result.append({'Features': concatenated_features})

df_car_fearures = pd.DataFrame(result)

CAR SPECIFICATION

In [10]:
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def extract_specs(specs):
   
    # Extract from 'top'
    top_data = extract_keys(specs.get('top', []))
    
    # Extract from 'list' in 'data'
    list_data = {}
    if 'data' in specs:
        for section in specs['data']:
            list_data.update(extract_keys(section.get('list', [])))
    
    # Combine both dictionaries
    combined_data = {**top_data, **list_data}
    return combined_data

car_spec_df = df["new_car_specs"].apply(extract_specs).apply(pd.Series)

In [11]:
car_spec_df.drop(columns=["Ground Clearance Unladen","Seats","Values per Cylinder","Front Tread","Seating Capacity",
                          "Compression Ratio","BoreX Stroke","Gross Weight","Alloy Wheel Size","Rear Tread","Acceleration",
                          "Top Speed","Wheel Size","Value Configuration"
                          ],inplace=True)

def millimeter(value,mini):
    if pd.notna(value) and mini in value:
        value = value.replace(mini, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

def meter(value,mini1,mini2,mini3):
    if pd.notna(value) and mini1 and mini2 and mini3 in value:
        value = value.replace(mini1, "").replace(mini2, "").replace(mini3, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

car_spec_df["Mileage"] = car_spec_df["Mileage"].str.replace(" kmpl", "", regex=True)
car_spec_df["Mileage"] = pd.to_numeric(car_spec_df["Mileage"], errors='coerce')

car_spec_df["Torque"] = car_spec_df["Torque"].str.replace("Nm", "", regex=True).str.replace("nm", "", regex=True)
car_spec_df["Torque"] = pd.to_numeric(car_spec_df["Torque"], errors='coerce')

car_spec_df["Engine"] = car_spec_df["Engine"].str.replace("CC", "", regex=True)
car_spec_df["Engine"] = pd.to_numeric(car_spec_df["Engine"], errors='coerce')

car_spec_df["Displacement"]=pd.to_numeric(car_spec_df["Displacement"], errors='coerce')

car_spec_df["No of Cylinder"] = pd.to_numeric(car_spec_df["No of Cylinder"], errors='coerce')

car_spec_df["No Door Numbers"]=pd.to_numeric(car_spec_df["No Door Numbers"], errors='coerce')

car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].apply(lambda x: millimeter(x, "-litre"))

car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].apply(lambda x:meter(x, "metres","mtrs.","m"))


In [12]:
car_spec_df["Mileage"]=car_spec_df["Mileage"].fillna(19.7)
car_spec_df["Torque"]=car_spec_df["Torque"].fillna(1370)
car_spec_df["No of Cylinder"]=car_spec_df["No of Cylinder"].fillna(4.0)
car_spec_df["Length"]=car_spec_df["Length"].fillna(	4047.6)
car_spec_df["Width"]=car_spec_df["Width"].fillna(1709.4)
car_spec_df["Height"]=car_spec_df["Height"].fillna(1567.4)
car_spec_df["Wheel Base"]=car_spec_df["Wheel Base"].fillna(2523.9)
car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].fillna(5.0)
car_spec_df["No Door Numbers"]=car_spec_df["No Door Numbers"].fillna(5.0)
car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].fillna(355.1)

In [13]:
df = pd.concat([new_car_details_df, car_overview, df_car_fearures, car_spec_df], axis=1)

In [14]:
df

,state,ft,bt,km,transmission,ownerNo,oem,model,modelYear,centralVariantId,...,Kerb Weight,Gear Box,Drive Type,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,No Door Numbers,Cargo Volumn
0,hyderabad,Petrol,Hatchback,"20,000",Automatic,1,Volkswagen,Volkswagen Polo,2022,7746,...,1106,6 Speed,FWD,Electronic,4.90,Disc,Drum,"Tubeless,Radial",5,280.0
1,hyderabad,Petrol,Hatchback,"85,582",Manual,2,Hyundai,Hyundai EON,2014,1502,...,880,5 Speed,FWD,Power,4.60,Disc,Drum,Tubeless,5,355.1
2,hyderabad,Petrol,SUV,"10,000",Manual,1,Hyundai,Hyundai Venue,2023,9153,...,1106,5-Speed,NaN,Power,5.00,Disc,Drum,"Tubeless, Radial",5,355.1
3,hyderabad,Petrol,Hatchback,"50,750",Manual,1,Maruti,Maruti Baleno,2017,3970,...,890Kg,5 Speed,FWD,Power,5.00,Disc,Drum,"Tubeless,Radial",5,355.1
4,hyderabad,Petrol,SUV,"10,000",Automatic,1,Mahindra,Mahindra Thar,2022,8074,...,1825,6 Speed,4X4,Power,5.00,Disc,Drum,"Radial, Tubeless",3,355.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,hyderabad,Diesel,SUV,"60,000",Manual,1,Mahindra,Mahindra XUV500,2013,3823,...,1940,6 Speed,FWD,Power,5.60,Disc & Caliper Type,Disc & Caliper Type,"Tubeless,Radial",5,355.1
1479,hyderabad,Diesel,Sedan,"50,000",Automatic,1,Jaguar,Jaguar XF,2014,5007,...,1780kg,8 Speed,RWD,Power,5.75,Disc,Disc,"Tubeless,Radial",4,355.1
1480,hyderabad,Diesel,Sedan,"1,20,000",Automatic,3,BMW,BMW 5 Series,2018,250,...,1660,8-Speed Steptronic,RWD,Power,5.60,Ventilated Disc,Ventilated Disc,"Tubeless, Runflat",4,355.1
1481,hyderabad,Diesel,SUV,"50,000",Automatic,2,Tata,Tata Nexon,2019,6500,...,1305Kg,6 Speed,FWD,Power,5.10,Disc,Drum,Tubeless Radial Tyres,5,350.0
